In [35]:
import os
import warnings
import matplotlib
matplotlib.use('Agg')
from matplotlib import figure
from matplotlib.backends import backend_agg
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
warnings.simplefilter(action='ignore')
import seaborn as sns
tfd = tfp.distributions
from utils import *

In [36]:
NUM_CLASSES = 10
LEARNING_RATE = 0.001
NUM_EPOCHS = 300
BATCH_SIZE =128

In [40]:
train_set, heldout_set = tf.keras.datasets.mnist.load_data()
labels = pd.Series(train_set[1])
print("Training set class count")
labels.value_counts()

1    6742
7    6265
3    6131
2    5958
9    5949
0    5923
6    5918
8    5851
4    5842
5    5421
dtype: int64

In [41]:
where_in = np.where(train_set[1] != 3)
where_in
train_in = train_set[0][where_in]
train_set_in = (train_set[0][where_in], train_set[1][where_in])
train_in.shape

(array([    0,     1,     2, ..., 59997, 59998, 59999], dtype=int64),)

In [48]:
def create_model():

  model = tf.keras.models.Sequential([
      tf.keras.layers.Convolution2D(
          6, kernel_size=5, padding='SAME',
          activation=tf.nn.relu),
      tf.keras.layers.MaxPooling2D(
          pool_size=[2, 2], strides=[2, 2],
          padding='SAME'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Convolution2D(
          16, kernel_size=5, padding='SAME',
          activation=tf.nn.relu),
      tf.keras.layers.MaxPooling2D(
          pool_size=[2, 2], strides=[2, 2],
          padding='SAME'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Convolution2D(
          120, kernel_size=5, padding='SAME',
          activation=tf.nn.relu),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(
          84,
          activation=tf.nn.relu),
      tf.keras.layers.Dense(
          NUM_CLASSES,
          activation=tf.nn.softmax)
  ])


  optimizer = tf.keras.optimizers.Adam(lr= LEARNING_RATE)
  model.compile(optimizer, loss='categorical_crossentropy',
                metrics=['accuracy'], experimental_run_tf_function=False)
  return model

model = create_model()
model.build(input_shape=[None, 28, 28, 1])
train_seq = DatasetSequence(data=train_set_in, batch_size=BATCH_SIZE)
heldout_seq = DatasetSequence(data=heldout_set, batch_size=BATCH_SIZE)

In [49]:
num_epochs = 300
num_monte_carlo = 10
print(' ... Training convolutional neural network')
for epoch in range(num_epochs):
    print(epoch)
    epoch_accuracy, epoch_loss = [], []
    for step, (batch_x, batch_y) in enumerate(train_seq):
      batch_loss, batch_accuracy = model.train_on_batch(batch_x, batch_y)
model.save('mnist_nonbayes.h5')

 ... Training convolutional neural network
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266